In [13]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

geojson_dir = 'D:/datascienEO'
all_data = []

for file_name in os.listdir(geojson_dir):
    if file_name.endswith('.geojson'):
        geojson_file = os.path.join(geojson_dir, file_name)
        geo_data = gpd.read_file(geojson_file)  # Read the GeoJSON file
        all_data.append(geo_data)

# Combine all GeoDataFrames into one
merged_geo_data = gpd.GeoDataFrame(pd.concat(all_data, ignore_index=True))

print(merged_geo_data.head())


Empty GeoDataFrame
Columns: [geometry]
Index: []


In [23]:
# Extract the coordinates and species
merged_geo_data['coordinates'] = merged_geo_data['geometry'].apply(lambda x: (x.x, x.y))  # Assuming the geometry is a Point
merged_geo_data['species'] = merged_geo_data['species']  # Replace 'l3_species' with the actual label column name

Index(['geometry', 'coordinates'], dtype='object')


KeyError: 'object'

In [27]:
numerical_features = merged_geo_data[['NDVI', 'EVI', 'EVI2', 'SAVI', 'NDWI']]

# Standardize the data (PCA is sensitive to the scale of features)
scaler = StandardScaler()
numerical_features_scaled = scaler.fit_transform(numerical_features)


pca = PCA(n_components=2)  # Reduce to 2 components, you can adjust based on your needs
pca_components = pca.fit_transform(numerical_features_scaled)


merged_geo_data['PCA_1'] = pca_components[:, 0]
merged_geo_data['PCA_2'] = pca_components[:, 1]


print(f"Explained Variance Ratio: {pca.explained_variance_ratio_}")

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.scatter(merged_geo_data['PCA_1'], merged_geo_data['PCA_2'], c='blue', alpha=0.5)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('PCA of Vegetation Indices')
plt.show()

KeyError: "None of [Index(['NDVI', 'EVI', 'EVI2', 'SAVI', 'NDWI'], dtype='object')] are in the [columns]"

In [15]:
def extract_s2_data_for_point(coordinate):
    # Replace with your method for extracting Sentinel-2 image patches
    return extract_patch_from_coordinates(coordinate)  # Custom function to load patch

image_patches = []
labels = []

for index, row in merged_geo_data.iterrows():
    species = row['species']
    coordinates = row['coordinates']
    patch = extract_s2_data_for_point(coordinates)
    image_patches.append(patch)
    labels.append(species)

image_patches = np.array(image_patches)
labels = np.array(labels)


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

le = LabelEncoder()
labels = le.fit_transform(labels)

labels = tf.keras.utils.to_categorical(labels)

X_train, X_val, y_train, y_val = train_test_split(image_patches, labels, test_size=0.2, random_state=42)


ValueError: zero-size array to reduction operation maximum which has no identity

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 13)),  # Adjust input shape based on your data
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')  # Number of classes: tree species
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)


D:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'X_train' is not defined

In [25]:
import json

# Load one of your GeoJSON files
with open('D:/datascienEO/broadleaf_beech_european beech.geojson', 'r') as f:
    geojson_data = json.load(f)

# Print the structure to understand where the species data is located
print(json.dumps(geojson_data, indent=2))


{
  "type": "FeatureCollection",
  "features": []
}
